In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
from nltk.stem import WordNetLemmatizer

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import urllib
from io import open
from importlib import reload
import paho.mqtt.client as mqtt
import paho.mqtt.subscribe as subscribe
import time
import json
from collections import namedtuple
import import_ipynb
from importlib import reload

import sys
reload(sys)
#sys.setdefaultencoding('ISO-8859-1')
# In[2]:


# title = "stories"
alpha = 0.3



dataset = []

c = False


text = urllib.request.urlopen("https://storage.googleapis.com/knowledge-base-data/index.html").read().decode(
    'utf-8').strip()

# file.close()
file_name = re.findall('><A HREF="(.*)">', text)
file_title = re.findall('<BR><TD> (.*)\n', text)

if c == False:
    file_name = file_name[2:]
    c = True

for j in range(len(file_name)):
    dataset.append(("" + str(file_name[j]), file_title[j]))

print(dataset)

# In[21]:


len(dataset)

# In[22]:


N = len(dataset)
print("len dataset:", N)


# In[23]:


def print_doc(id):
    print("dataset[id]:", dataset[id])
    text = urllib.request.urlopen(dataset[id][0]).read().decode('ISO-8859-1').strip()

    # file = open(dataset[id][0], 'r', encoding='cp1250')
    # text = file.read().strip()
    # file.close()
    # print(text)


# In[24]:


def convert_lower_case(data):
    return np.char.lower(data)


# In[25]:


def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text


# In[26]:


def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data


# In[27]:


def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


# In[28]:


def stemming(data):
    stemmer = PorterStemmer()

    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text


# In[29]:


def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    try:
        for w in tokens:
            try:
                w = num2words(int(w))
            except:
                a = 0
            new_text = new_text + " " + w
        new_text = np.char.replace(new_text, "-", " ")

    except:
        print ("Exception .....")
    return new_text


nltk.download('wordnet')


def lemmatizing_words(data):
    lemmatizer = WordNetLemmatizer()

    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
    return new_text


# In[30]:


def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data)  # remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data)  # needed again as we need to stem the words
    data = remove_punctuation(data)  # needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data)  # needed again as num2word is giving stop words 101 - one hundred and one
    data = lemmatizing_words(data)
    return data


# In[32]:


processed_text = []
processed_title = []
for i in dataset[:N]:
    # file = open(i[0], 'r', encoding="utf8", errors='ignore')

    # text = file.read().strip()
    text = urllib.request.urlopen(i[0]).read().decode('ISO-8859-1').strip()

    # file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

total_vocab_size = len(DF)

print ("total_vocab_size:", total_vocab_size)

total_vocab = [x for x in DF]

print(total_vocab[:20])


def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c


doc = 0

tf_idf = {}

for i in range(N):

    tokens = processed_text[i]

    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])

    for token in np.unique(tokens):
        tf = counter[token] / words_count
        df = doc_freq(token)
        idf = np.log((N + 1) / (df + 1))

        tf_idf[doc, token] = tf * idf

    doc += 1

doc = 0

tf_idf_title = {}

for i in range(N):

    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        tf = counter[token] / words_count
        df = doc_freq(token)
        idf = np.log((N + 1) / (df + 1))  # numerator is added 1 to avoid negative values

        tf_idf_title[doc, token] = tf * idf

    doc += 1

print("tf_idf:", tf_idf[(0, "renaud")])


print("th_idf_title:", tf_idf_title[(0, "renaud")])


for i in tf_idf:
    tf_idf[i] *= alpha


for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]


print("len tf_idf:", len(tf_idf))


def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    query_weights = {}
    for key in tf_idf:
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    print("")
    l = []
    for i in query_weights[:10]:
        l.append(i[0])

    print ("l:", l)
    # print_doc(l[0])
    print(dataset[l[1]][0])
    s = ""
    s = dataset[l[0]][0]+",  "+dataset[l[1]][0]+", "+dataset[l[2]][0]
    print ('**s:',s)
    return s;


#matching_score(10, "Power Outage")



def cosine_sim(a, b):
    cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return cos_sim


# In[51]:


D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass


def gen_vector(tokens):
    Q = np.zeros((len(total_vocab)))

    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}

    for token in np.unique(tokens):

        tf = counter[token] / words_count
        df = doc_freq(token)
        idf = math.log((N + 1) / (df + 1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf * idf
        except:
            pass
    return Q


# In[55]:


def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("\nQuery:", query)
    print("")
    print(tokens)

    d_cosines = []

    query_vector = gen_vector(tokens)

    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))

    out = np.array(d_cosines).argsort()[-k:][::-1]

    print("")

    print(out)



D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass


FAULT_EXTRACTED_DATA_TOPIC = "fault-extracted-data-events"
BOT_EXTRACTED_FAULT_EVENTS_TOPIC = "bot-extracted-fault-events"
BOT_RESPONSE_EVENTS_TOPIC = "bot-fault-events-sub"

def on_message(client, userdata, message):
    print("***** %s ***** %s" % (message.topic, message.payload))
    data = message.payload
    x = json.loads(data, object_hook=lambda d: namedtuple('X', d.keys())(*d.values()))
    print (x.deviceId, x.fault, x.topic)
    if (x.topic == "nlp-fault-events"):
        print("%%%%received message to topic nlp-fault-events and insering in database%%%")
        print("Matching simillarity called for cosine_simillarity")
        #Q = cosine_similarity(10, x.fault)
        l =  matching_score(10,x.fault)
        # printing the list using loop
        print("first matched topic:")
        print("%%% l is:",l)
        for x in range(len(l)):
            print_doc(l[x]),
            print ("##############################################################")


    elif (x.topic == "bot-fault-events"):
        print("Publishing message to topic", "chatbot-fault-events")
        l = matching_score(10, x.fault)
        # printing the list using loop
        print("first matched topic:")
        print("%%% l is:", l)
        payloadMsg = l
        client.publish(BOT_RESPONSE_EVENTS_TOPIC, payloadMsg, qos=2)
        print("message published on bot-fault-events-sub")

def on_log(client, userdata, level, buf):
    #print("log: ",buf)
    print (".")

############################################

broker_address="192.168.1.3"
print("creating new instance")
client = mqtt.Client("P2") #create new instance
#client = mqtt.Client()
client.on_message = on_message#attach function to callback
client.on_log=on_log

print("connecting to broker")
client.connect(broker_address, 1883, 60)#connect to broker

print("Subscribing to topic","fault-extracted-data-events")
message = client.subscribe(FAULT_EXTRACTED_DATA_TOPIC, qos=1)
print("Subscribing to topic","fault-extracted-data-events")
message = client.subscribe(BOT_EXTRACTED_FAULT_EVENTS_TOPIC, qos=1)

print ("Message received")

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()


[('https://storage.googleapis.com/knowledge-base-data/sre01.txt', 'SRE: The Saga Of The Best SRE Game Ever Played! By Josh Renaud'), ('https://storage.googleapis.com/knowledge-base-data/srex.txt', 'Solar Realms Elite: X1 and X2, by Josh Renaud'), ('https://storage.googleapis.com/knowledge-base-data/100west.txt', 'Going 100 West by 53 North by Jim Prentice (1990)'), ('https://storage.googleapis.com/knowledge-base-data/20.lws', 'What Research and Development Was Always Meant to Be, by M. Peshota'), ('https://storage.googleapis.com/knowledge-base-data/3gables.txt', 'The Adventure of the Three Gables'), ('https://storage.googleapis.com/knowledge-base-data/3lpigs.txt', 'The Story of the 3 Little Pigs'), ('https://storage.googleapis.com/knowledge-base-data/3sonnets.vrs', 'A Collection of Sonnets by Staeorra Rokraven (March 9, 1989)'), ('https://storage.googleapis.com/knowledge-base-data/3student.txt', 'The Adventure of the Three Students'), ('https://storage.googleapis.com/knowledge-base-dat

[nltk_data] Downloading package wordnet to /home/dnilesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


total_vocab_size: 9668
['sre', 'saga', 'best', 'game', 'ever', 'play', 'josh', 'renaud', 'bbse', 'call', 'solar', 'realm', 'elit', 'devolop', 'ibm', 'version', 'atari', 'see', 'realli', 'fun']
tf_idf: 0.004844772152299886
th_idf_title: 0.004844772152299886
len tf_idf: 35330
creating new instance
connecting to broker
.
Subscribing to topic fault-extracted-data-events
.
Subscribing to topic fault-extracted-data-events
.
Message received
.
.
.
.
.
***** bot-extracted-fault-events ***** b'{"topic": "bot-fault-events", "fault": "Short circuit Fault \\n", "deviceId": "BD Transport \\n"}'
BD Transport 
 Short circuit Fault 
 bot-fault-events
Publishing message to topic chatbot-fault-events
Matching Score

Query: Short circuit Fault 


['short', 'circuit', 'fault']

l: [41, 40, 42, 43, 3, 2, 14, 33, 18, 4]
https://storage.googleapis.com/knowledge-base-data/3arcingfault.txt
**s: https://storage.googleapis.com/knowledge-base-data/shortCircuit.txt,  https://storage.googleapis.com/knowledge-base-d

KeyboardInterrupt: 